In [1]:
import numpy as np
import json
from tempfile import TemporaryFile
import sys
from numpy import genfromtxt
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from array import *
import random
import array

## Defines FUNCTIONS

In [2]:
def act(weights, aVec, nInput, nOutput, inPattern):
  # Turn weight vector into weight matrix
  if np.ndim(weights) < 2:
      nNodes = int(np.sqrt(np.shape(weights)[0]))
      wMat = np.reshape(weights, (nNodes, nNodes))
  else:
      nNodes = np.shape(weights)[0]
      wMat = weights

  # Vectorize input
  if np.ndim(inPattern) > 1:
      nSamples = np.shape(inPattern)[0]
  else:
      nSamples = 1

  # Run input pattern through ANN    
  nodeAct  = np.zeros((nSamples,nNodes))
  nodeAct[:,0] = 1 # Bias activation
  nodeAct[:,1:nInput+1] = inPattern

  # Propagate signal through hidden to output nodes
  iNode = nInput+1
  for iNode in range(nInput+1,nNodes):
      rawAct = np.dot(nodeAct, wMat[:,iNode]).squeeze()
      nodeAct[:,iNode] = applyAct(aVec[iNode], rawAct)
      # print(nodeAct) # ADDED ADDED ADDED
  return nodeAct
#return nodeAct[:,-nOutput:]

In [3]:
def applyAct(actId, x):
  '''
  case 1  -- Linear
  case 2  -- Unsigned Step Function
  case 3  -- Sin
  case 4  -- Gausian with mean 0 and sigma 1
  case 5  -- Hyperbolic Tangent (signed)
  case 6  -- Sigmoid unsigned [1 / (1 + exp(-x))]
  case 7  -- Inverse
  case 8  -- Absolute Value
  case 9  -- Relu
  case 10 -- Cosine

  '''
  if actId == 1:   # Linear
    value = x

  if actId == 2:   # Unsigned Step Function
    value = 1.0*(x>0.0)
    #value = (np.tanh(50*x/2.0) + 1.0)/2.0

  elif actId == 3: # Sin
    value = np.sin(np.pi*x) 

  elif actId == 4: # Gaussian with mean 0 and sigma 1
    value = np.exp(-np.multiply(x, x) / 2.0)

  elif actId == 5: # Hyperbolic Tangent (signed)
    value = np.tanh(x)     

  elif actId == 6: # Sigmoid (unsigned)
    value = (np.tanh(x/2.0) + 1.0)/2.0

  elif actId == 7: # Inverse
    value = -x

  elif actId == 8: # Absolute Value
    value = abs(x)   
    
  elif actId == 9: # Relu
    value = np.maximum(0, x)   

  elif actId == 10: # Cosine
    value = np.cos(np.pi*x)
    
  else:
    value = x

  return value

In [4]:
def importNet(fileName):
  ind = np.loadtxt(fileName, delimiter=',')
  wMat = ind[:,:-1]     # Weight Matrix
  aVec = ind[:,-1]      # Activation functions

  # Create weight key (WHY IS THERE A WEIGHT KEY)
  wVec = wMat.flatten()
  wVec[np.isnan(wVec)]=0
  wKey = np.where(wVec!=0)[0] 

  return wVec, aVec, wKey

## Import NET

In [5]:
a = importNet('log/test_best.out')
weights = a[0]
weights = weights.flatten()
activations = a[1]
activations = activations.flatten()
weightkey = a[2]
weightkey = weightkey.flatten()

#np.set_printoptions(threshold=sys.maxsize)
#print(weights)
#print(activations)
#print(weightkey)

In [6]:
weight_adjust = json.load(open('log_tool/biped.cma.4.360.best.json', 'r'))
weight_adjust = weight_adjust[0]
weight_adjust = np.array(weight_adjust)

for i in range(0,len(weightkey)):
    x = weightkey[i]
    weights[x] = weight_adjust[i]

#print(weights)

### Return the degree and neighbors of each node

## Test NET ACTIVITY

In [7]:
#with open('sim_input.txt') as f:
#    lines = f.readlines()
#print(lines[0])

input_list = []

def read_lists():
    with open('sim_input.txt') as file:
        sublist = []
        previous_line = ''
        for line in file:
            line = line.strip()
            if line.startswith('[') and previous_line.endswith(']'):
                yield sublist
                sublist = []
            sublist.append(line)
            previous_line = line
        yield sublist

for sublist in read_lists():
    input_list.append(sublist)
    
# print(input_list[0])
# print(input_list[1])

In [8]:
input_list_final = []

for element in input_list:
    x = element[0]
    x = x.strip('][').split(' ')
    x = list(filter(None, x))
    x = x[:14]
    x = [float(i) for i in x]
    input_list_final.append(x)

#print(input_list_final[0])
#print(input_list_final[1])

In [9]:
activity_list_final = pd.DataFrame()

for element in input_list_final:
    activity = act(weights, activations, 14, 4, element)
    activity = activity.tolist()
    #activity = activity[0]
    activity_list_final = activity_list_final.append(activity)
    
print(activity_list_final)

    0        1         2         3       4       5             6       7    \
0   1.0  0.00275  0.000007 -0.000853 -0.0160  0.0924  1.980000e-03  0.8600   
0   1.0  0.03000  0.070000  0.010000 -0.0100  0.4100  5.000000e-02  0.0800   
0   1.0  0.10000  0.130000  0.050000 -0.0300  0.3300 -1.000000e+00 -0.0300   
0   1.0  0.16000  0.130000  0.050000 -0.0500  0.2400 -1.000000e+00 -0.1400   
0   1.0  0.23000  0.130000  0.050000 -0.0700  0.1600 -1.000000e+00 -0.2600   
..  ...      ...       ...       ...     ...     ...           ...     ...   
0   1.0  0.25400  0.068300  0.422000  0.0754 -0.8350  3.280000e-07 -0.0953   
0   1.0  0.30000  0.091800  0.424000  0.0749 -0.8350  1.190000e-07 -0.2130   
0   1.0  0.36000  0.120000  0.430000  0.0700 -0.8300  0.000000e+00 -0.3300   
0   1.0  0.42200  0.128000  0.429000  0.0540 -0.8350  2.640000e-04 -0.4470   
0   1.0  0.48500  0.126000  0.429000  0.0350 -0.8350  2.380000e-07 -0.5630   

         8    9    ...       202       203       204       205 

## EXPORT DATA

In [10]:
activity_list_final.to_csv('walk_2-28-21.csv', index=False)